<a href="https://colab.research.google.com/github/glitchheadgit/smilesChessFraud/blob/main/chessFraud_Egor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and imports

In [1]:
!pip install transformers datasets bertviz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 366.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm, trange

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
ds = pd.read_parquet("part-00000.parquet")
ds.head()

,Date,Round,White,Black,Result,Tournament,CurrentPosition,ECO,ECOUrl,WhiteElo,...,StartTime,EndDate,EndTime,Link,Moves,Moves_length,Tournament_long,Moves_clean,sf_n1_d22,sf_n10_d18
0,2020.07.18,-,Hikaru,imastraw,1-0,,2r3k1/N2q3p/b4pp1/3p4/3Bn3/1P2P3/PQ3PPP/2R3K1 ...,A04,Reti-Opening-Nimzo-Larsen-Variation-2...b6-3.B...,3263,...,20:15:48,2020.07.18,20:19:12,5169704302,1. Nf3 {[%clk 0:02:59.9]} 1... Nf6 {[%clk 0:02...,47,,"[Nf3, Nf6, b3, b6, Bb2, Bb7, d4, g6, e3, Bg7, ...","[[{'_1': 'e2e4', '_2': 52.0}], [{'_1': 'e7e6',...","[[{'_1': 'e2e4', '_2': 46.0}, {'_1': 'd2d4', '..."
1,2021.01.25,-,Bigfish1995,Azerichess,0-1,,6k1/5p1p/1B1p4/r5p1/P4r2/3K4/2P4P/R7 b - -,B33,Sicilian-Defense-Open-3...cxd4-4.Nxd4-Nf6-5.Nc3,3016,...,20:53:33,2021.01.25,21:00:11,6345643292,1. e4 {[%clk 0:03:01]} 1... c5 {[%clk 0:03:01]...,63,,"[e4, c5, Nf3, Nc6, d4, cxd4, Nxd4, Nf6, Nc3, e...","[[{'_1': 'e2e4', '_2': 52.0}], [{'_1': 'e7e6',...","[[{'_1': 'e2e4', '_2': 46.0}, {'_1': 'd2d4', '..."
2,2022.08.16,-,Grischuk,krzyzan94,1/2-1/2,titled-tuesday,8/5b2/5P2/8/8/1k6/p7/K7 w - -,E60,Kings-Indian-Defense,3041,...,21:30:45,2022.08.16,21:39:24,54424259185,1. d4 {[%clk 0:03:00.9]} 1... Nf6 {[%clk 0:02:...,156,late-titled-tuesday-blitz-august-16-2022-3304769,"[d4, Nf6, c4, g6, Bg5, Bg7, Nc3, h6, Bd2, d6, ...","[[{'_1': 'e2e4', '_2': 52.0}], [{'_1': 'd7d5',...","[[{'_1': 'e2e4', '_2': 46.0}, {'_1': 'd2d4', '..."
3,2020.03.02,-,Hikaru,Robert_Chessmood,1/2-1/2,,8/8/8/5p2/5B2/5K1k/8/8 b - -,D10,Slav-Defense...5.Bf4-Nc6-6.e3-Bf5,3239,...,15:50:01,2020.03.02,15:56:08,4553851201,1. d4 {[%clk 0:02:59.9]} 1... d5 {[%clk 0:02:5...,159,,"[d4, d5, c4, c6, Nc3, Nf6, cxd5, cxd5, Bf4, Nc...","[[{'_1': 'e2e4', '_2': 52.0}], [{'_1': 'd7d5',...","[[{'_1': 'e2e4', '_2': 46.0}, {'_1': 'd2d4', '..."
4,2019.02.07,-,wonderfultime,Hikaru,1/2-1/2,,8/6K1/4k1n1/6P1/5n2/8/8/8 w - -,A46,Indian-Game-Polish-Variation,2884,...,15:10:36,2019.02.07,15:17:12,3439963952,1. d4 {[%clk 0:02:59.9]} 1... Nf6 {[%clk 0:02:...,182,,"[d4, Nf6, Nf3, b5, Bf4, Bb7, Nbd2, e6, e3, a6,...","[[{'_1': 'e2e4', '_2': 52.0}], [{'_1': 'd7d5',...","[[{'_1': 'e2e4', '_2': 46.0}, {'_1': 'd2d4', '..."
